In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
import pdb
import _pickle as cPickle
import time
import subprocess
import glob

import random
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict, Counter
from ncrp import Topic, Doc, init, sample, get_perplexity, get_topic_specialization, get_hierarchical_affinities, get_freq_tokens_ncrp, get_docs, get_sum_cnt_words
from configure import get_config

# load config & data 

In [3]:
config = get_config(nb_name)
np.random.seed(config.seed)
random.seed(config.seed)

In [4]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

Exception ignored in: <_io.FileIO name='data/20news/instances.pkl' mode='rb' closefd=True>


In [5]:
config.n_doc = len(instances_train)
config.n_vocab = len(bow_idxs)
config.n_doc, config.n_vocab

(11258, 1995)

# run

## initialize log

In [6]:
checkpoint = []
ppl_min = np.inf
epoch = 0

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','VALID:','TEST:','SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','PPL','PPL', 'PPL','1', '2', '3', 'CHILD', 'OTHER']]))))

def update_checkpoint(config, checkpoint, epoch):
    checkpoint.append(config.path_model + '-%i' % epoch)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0)
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize data

In [7]:
topic_root = Topic(idx='0', sibling_idx=0, parent=None, depth=0, config=config)
train_docs = get_docs(instances_train, config)
dev_docs = get_docs(instances_dev, config)
test_docs = get_docs(instances_test, config)
init(train_docs, dev_docs, test_docs, topic_root)

0 10000 0 0 

## run

In [ ]:
time_start = time.time()
while epoch < config.n_epochs:
    sample(train_docs, dev_docs, test_docs, topic_root)
    ppl_train = get_perplexity(train_docs, topic_root)
    ppl_dev = get_perplexity(dev_docs, topic_root)
    
    if ppl_dev < ppl_min:
        ppl_min = ppl_dev
        ppl_test = get_perplexity(test_docs, topic_root)
        cPickle.dump([test_docs, topic_root], open(config.path_model + '-%i'%epoch, 'wb'))
        update_checkpoint(config, checkpoint, epoch)
        
    depth_spec = get_topic_specialization(test_docs, topic_root)
    hierarchical_affinities = get_hierarchical_affinities(topic_root)
    
    clear_output()
    time_log = int(time.time() - time_start)
    time_start = time.time()
    log_series = pd.Series([time_log, epoch, 0, \
            '%.0f'%ppl_train, '%.0f'%ppl_dev, '%.0f'%ppl_test, \
            '%.2f'%depth_spec[1], '%.2f'%depth_spec[2], '%.2f'%depth_spec[3], \
            '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
            index=log_df.columns)
    log_df.loc[epoch] = log_series    
    display(log_df)
    get_freq_tokens_ncrp(topic_root, idx_to_word, bow_idxs)
    
    cPickle.dump(log_df, open(config.path_log, 'wb'))
    epoch += 1

TRAIN: VALID: TEST: SPEC:             HIER:      
  Time Ep Ct    PPL    PPL   PPL     1     2     3 CHILD OTHER
0  385  0  0   1113   1103  1102  0.01  0.34  0.45  0.88  0.88
1  407  1  0   1091   1079  1077  0.02  0.38  0.45  0.91  0.91
2  407  2  0   1068   1055  1055  0.02  0.39  0.45  0.90  0.90
3  432  3  0   1046   1034  1034  0.02  0.41  0.45  0.92  0.92
4  458  4  0   1029   1019  1019  0.02  0.42  0.45  0.93  0.93
5  451  5  0   1015   1007  1006  0.03  0.42  0.45  0.92  0.92
6  454  6  0   1005    998   998  0.03  0.42  0.45  0.92  0.92
7  453  7  0    997    993   992  0.03  0.42  0.45  0.91  0.91
8  452  8  0    991    989   989  0.04  0.42  0.45  0.91  0.91
9  452  9  0    988    987   987  0.04  0.41  0.45  0.91  0.90

Exception ignored in: <_io.FileIO name='model/20news/ncrp/20news-mncrp-alp10.50.1-eta50-gam0.01-epoch50/log' mode='wb' closefd=True>


 0 11258 608051.0 use write get one article like know make go think
   0-1 8565 324558.0 people say one god write think go know make article
     0-1-1 8565 4223.0 screw weight patient go true la chapter stop wave close
   0-2 528 15927.0 pt vs la period team play gm hockey van pp
     0-2-1 528 393.0 city rangers nhl penalty game west fan ice bay bmw
   0-3 386 3060.0 db mov bh byte cs bit int oname al printf
     0-3-1 386 238.0 link cross edge visual unit window cs printer screen frame
   0-4 967 28589.0 space launch satellite mission nasa april national center orbit technology
     0-4-1 967 646.0 xterm nuclear plant ken engineer water export plan moon sale
   0-5 811 5325.0 wire ground outlet wiring circuit neutral connect electrical cable box
     0-5-1 811 483.0 cpu lock connect printer voice hence macintosh hp circuit plug
   0-6 1 0.0 bhj series ram dave knowledge prevent james throw washington legal
     0-6-1 1 0.0 bhj series ram dave knowledge prevent james throw washington